# Football Match Prediction - Clean Model Building

Saubere Implementation mit RandomForest und umfassendem Hyperparameter-Tuning.


In [1]:
import pickle
import pandas as pd
import numpy as np

# Load data
match_df = pickle.load(open("data/prepped_match_df.pck", "rb"))
print(f"Dataset loaded: {match_df.shape}")
match_df.head()


Dataset loaded: (4841, 69)


,teamHomeId,teamAwayId,matchDay,season,teamHomeValue,teamAwayValue,teamHomeRank,teamAwayRank,teamHomeAvgScoredGoals1,teamHomeAvgGottenGoals1,...,teamAwayAvgScoredGoals8,teamAwayAvgGottenGoals8,teamAwayAvgTeamPoints8,teamAwayAvgScoredGoals9,teamAwayAvgGottenGoals9,teamAwayAvgTeamPoints9,teamAwayAvgScoredGoals10,teamAwayAvgGottenGoals10,teamAwayAvgTeamPoints10,resultClass
90,7,54,11,2009,109.40,61.98,10,18,1.0,1.0,...,0.625,2.625,0.125,0.666667,2.555556,0.111111,0.7,2.3,0.4,2:0
94,65,55,11,2009,59.13,50.88,13,11,0.0,0.0,...,1.500,1.250,1.375,1.444444,1.222222,1.333333,1.3,1.2,1.2,0:1
93,100,87,11,2009,136.28,55.00,2,15,3.0,3.0,...,0.625,1.750,0.500,0.777778,1.666667,0.777778,1.0,1.8,0.8,2:3
92,16,40,11,2009,124.65,279.55,14,5,0.0,1.0,...,1.875,0.875,2.000,1.777778,0.888889,1.888889,1.7,0.9,1.8,0:0
91,131,81,11,2009,155.43,48.78,7,8,0.0,0.0,...,1.500,1.375,1.875,1.444444,1.333333,1.777778,1.5,1.4,1.7,3:3


In [2]:
# Prepare target variable
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
match_df["resultClass"] = enc.fit_transform(match_df["resultClass"])


In [3]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    match_df.drop("resultClass", axis=1), 
    match_df["resultClass"], 
    test_size=0.2, 
    shuffle=False  # Time series data
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


Training set: (3872, 68)
Test set: (969, 68)


In [4]:
# Custom Kicktipp Scorer
from sklearn.metrics import make_scorer

def kicktipp_score(y_true, y_pred, **kwargs):
    """Custom scoring function for Kicktipp predictions"""
    # Convert to int64 for consistency
    y_true = y_true.astype('int64') if hasattr(y_true, 'astype') else np.array(y_true, dtype='int64')
    y_pred = y_pred.astype('int64') if hasattr(y_pred, 'astype') else np.array(y_pred, dtype='int64')
    
    # Decode predictions back to score format
    y_true_decoded = [x.split(":") for x in enc.inverse_transform(y_true)]
    y_pred_decoded = [x.split(":") for x in enc.inverse_transform(y_pred)]
    
    score_value = 0
    for true, pred in zip(y_true_decoded, y_pred_decoded):
        # Exact match: 5 points
        if true[0] == pred[0] and true[1] == pred[1]:
            score_value += 5
        # Goal difference correct: 3 points
        elif (int(true[0]) - int(true[1])) == (int(pred[0]) - int(pred[1])):
            score_value += 3
        # Winner correct: 1 point
        elif ((true[0] > true[1]) and (pred[0] > pred[1])) or \
             ((true[0] < true[1]) and (pred[0] < pred[1])) or \
             ((true[0] == true[1]) and (pred[0] == pred[1])):
            score_value += 1
    
    return round(score_value / (len(y_true)/306))  # Normalize to Kicktipp scale

kicktipp_scorer = make_scorer(kicktipp_score, greater_is_better=True)
print("✅ Custom Kicktipp scorer ready")

# Cross-Validation Setup
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
print("✅ Time Series Cross-Validation ready (5 splits)")

✅ Custom Kicktipp scorer ready
✅ Time Series Cross-Validation ready (5 splits)


In [5]:
# Baseline Model
from collections import Counter

# Always predict most frequent class
majority_class = Counter(y_train).most_common(1)[0][0]
baseline_predictions = np.full(len(y_test), majority_class)
baseline_score = kicktipp_score(y_test, baseline_predictions)

print(f"📊 Baseline (majority class): {baseline_score} points")


📊 Baseline (majority class): 298 points


# Random Forest

In [13]:
# RandomForest with Comprehensive Hyperparameter Tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

print("🌳 Setting up RandomForest with comprehensive hyperparameter search...")

# Comprehensive parameter grid
param_distributions = {
    'n_estimators': np.arange(100, 501, 25),           # 100, 125, 150, ..., 500
    'max_depth': [None] + list(np.arange(5, 31, 2)),   # None, 5, 7, 9, ..., 29
    'min_samples_split': np.arange(2, 21, 1),          # 2, 3, 4, ..., 20
    'min_samples_leaf': np.arange(1, 16, 1),           # 1, 2, 3, ..., 15
    'max_features': ['sqrt', 'log2', None] + list(np.arange(0.1, 1.0, 0.1)),  # sqrt, log2, None, 0.1-0.9
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes': [None] + list(np.arange(10, 101, 10))  # None, 10, 20, ..., 100
}

# Calculate total combinations
total_combinations = 1
for param, values in param_distributions.items():
    total_combinations *= len(values)

print(f"📊 Hyperparameter space: {total_combinations:,} total combinations")
print(f"📊 Search iterations: 150 ({150/total_combinations*100:.4f}% coverage)")

# RandomForest model
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# RandomizedSearchCV
rf_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=150,  # Comprehensive search
    cv=tscv,
    scoring=kicktipp_scorer,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("🚀 Starting hyperparameter optimization...")
rf_search.fit(X_train, y_train)

# Results
best_params = rf_search.best_params_
best_cv_score = rf_search.best_score_
best_model = rf_search.best_estimator_

print(f"\n🎯 Best Parameters: {best_params}")
print(f"🎯 Best CV Score: {best_cv_score:.1f} points")


🌳 Setting up RandomForest with comprehensive hyperparameter search...
📊 Hyperparameter space: 35,814,240 total combinations
📊 Search iterations: 150 (0.0004% coverage)
🚀 Starting hyperparameter optimization...
Fitting 5 folds for each of 150 candidates, totalling 750 fits


/Users/maxlautenbach/PycharmProjects/football_match_prediction/.venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



🎯 Best Parameters: {'n_estimators': np.int64(300), 'min_samples_split': np.int64(20), 'min_samples_leaf': np.int64(10), 'max_leaf_nodes': np.int64(80), 'max_features': 'log2', 'max_depth': np.int64(13), 'criterion': 'gini', 'bootstrap': False}
🎯 Best CV Score: 341.0 points


In [14]:
# Final Model Evaluation
predictions = best_model.predict(X_test)
final_score = kicktipp_score(y_test, predictions)

print("=" * 50)
print("🏆 FINAL RESULTS")
print("=" * 50)
print(f"📊 Baseline Score:        {baseline_score} points")
print(f"🌳 RandomForest CV Score: {best_cv_score:.1f} points")
print(f"🌳 RandomForest Test Score: {final_score} points")
print(f"📈 Improvement over baseline: +{final_score - baseline_score} points")
print("=" * 50)

if final_score > baseline_score:
    print(f"✅ SUCCESS: {final_score - baseline_score} points improvement!")
else:
    print(f"❌ Model performs worse than baseline by {baseline_score - final_score} points")


🏆 FINAL RESULTS
📊 Baseline Score:        298 points
🌳 RandomForest CV Score: 341.0 points
🌳 RandomForest Test Score: 333 points
📈 Improvement over baseline: +35 points
✅ SUCCESS: 35 points improvement!


In [15]:
# Save Model and Encoder
from datetime import datetime

# Generate timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save best model
model_filename = f"classifier_{timestamp}.pck"
pickle.dump(best_model, open(model_filename, "wb"))

# Save encoder (fixed filename)
encoder_filename = "encoder.pck"
pickle.dump(enc, open(encoder_filename, "wb"))

print(f"💾 Model saved as: {model_filename}")
print(f"💾 Encoder saved as: {encoder_filename}")
print(f"💾 Final score: {final_score} points")
print(f"💾 Best parameters: {best_params}")


💾 Model saved as: classifier_20250929_211007.pck
💾 Encoder saved as: encoder.pck
💾 Final score: 333 points
💾 Best parameters: {'n_estimators': np.int64(300), 'min_samples_split': np.int64(20), 'min_samples_leaf': np.int64(10), 'max_leaf_nodes': np.int64(80), 'max_features': 'log2', 'max_depth': np.int64(13), 'criterion': 'gini', 'bootstrap': False}


In [16]:
# Final Comparison: Original RF vs Stacking (Clean Version)
print("=" * 60)
print("🏆 FINAL MODEL COMPARISON")
print("=" * 60)
print(f"📊 Baseline Score:        {baseline_score} points")
print(f"🌳 Original RandomForest: {final_score} points")
print(f"🏗️ Stacking Ensemble:     {stacking_score} points")
print("=" * 60)

original_improvement = final_score - baseline_score
stacking_improvement = stacking_score - baseline_score
stacking_vs_original = stacking_score - final_score

print(f"📈 Original RF improvement:  +{original_improvement} points")
print(f"📈 Stacking improvement:     +{stacking_improvement} points")
print(f"🏗️ Stacking vs Original:     {stacking_vs_original:+} points")

if stacking_score > final_score:
    print(f"\n✅ STACKING WINS! Ensemble improved by {stacking_vs_original} points")
    best_final_model = stacking_classifier
    best_final_score = stacking_score
    best_approach = "Stacking"
else:
    print(f"\n🤔 Original RF still better by {-stacking_vs_original} points")
    best_final_model = best_model
    best_final_score = final_score
    best_approach = "Original RF"

print(f"\n🏆 WINNER: {best_approach} with {best_final_score} points")

# Komplexität vs Performance
print(f"\n💡 COMPLEXITY vs PERFORMANCE:")
print(f"  🌳 Original RF: {final_score:3.0f} points (1 model)")
print(f"  🏗️ Stacking:    {stacking_score:3.0f} points (3 models + meta-learner)")

if stacking_vs_original >= 3:
    print(f"\n🏆 RECOMMENDATION: Use Stacking Ensemble!")
    print(f"   ✅ Significant improvement: +{stacking_vs_original} points")
else:
    print(f"\n🏆 RECOMMENDATION: Use Original RandomForest!")
    print(f"   ✅ Simpler model with comparable performance")


🏆 FINAL MODEL COMPARISON
📊 Baseline Score:        298 points
🌳 Original RandomForest: 333 points
🏗️ Stacking Ensemble:     320 points
📈 Original RF improvement:  +35 points
📈 Stacking improvement:     +22 points
🏗️ Stacking vs Original:     -13 points

🤔 Original RF still better by 13 points

🏆 WINNER: Original RF with 333 points

💡 COMPLEXITY vs PERFORMANCE:
  🌳 Original RF: 333 points (1 model)
  🏗️ Stacking:    320 points (3 models + meta-learner)

🏆 RECOMMENDATION: Use Original RandomForest!
   ✅ Simpler model with comparable performance


In [17]:
# Save Best Model and Encoder (Clean Version)
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Determine which model performed better
if stacking_score > final_score:
    model_to_save = stacking_classifier
    score_to_save = stacking_score
    approach_name = "Stacking"
    model_type = "StackingClassifier with 3 RandomForest base models"
else:
    model_to_save = best_model
    score_to_save = final_score
    approach_name = "Original RF"
    model_type = "RandomForestClassifier"

model_filename = f"classifier_{timestamp}.pck"
pickle.dump(model_to_save, open(model_filename, "wb"))

# Save encoder
encoder_filename = "encoder.pck"
pickle.dump(enc, open(encoder_filename, "wb"))

print(f"💾 Best model ({approach_name}) saved as: {model_filename}")
print(f"💾 Model type: {model_type}")
print(f"💾 Encoder saved as: {encoder_filename}")
print(f"💾 Final score: {score_to_save} points")
print(f"💾 Improvement over baseline: +{score_to_save - baseline_score} points")


💾 Best model (Original RF) saved as: classifier_20250929_211007.pck
💾 Model type: RandomForestClassifier
💾 Encoder saved as: encoder.pck
💾 Final score: 333 points
💾 Improvement over baseline: +35 points
